In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH
from sklearn.ensemble import RandomForestRegressor
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math

In [2]:
train = pd.read_csv('data/trip_train.csv')

In [3]:
test = pd.read_csv('data/trip_test.csv')

In [4]:
weather = pd.read_csv('data/weather.csv')

In [9]:
station = pd.read_csv('data/station.csv')

In [6]:
#Preprocesing

In [10]:
stationReduced = station.loc[:,['id','name','city']]

def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

stationProcesed = stationReduced.drop(labels=['name','city'],axis=1)


In [11]:
stationProcesed.head()

,id,zip_code
0,2,95113
1,3,95113
2,4,95113
3,5,95113
4,6,95113


In [13]:
test = test.drop(labels=['zip_code',\
                         'end_date',\
                         'end_station_name',\
                         'end_station_id',\
                         'bike_id'],\
                 axis=1)

In [14]:
train = train.drop(labels=['zip_code',\
                           'end_date',\
                           'end_station_name',\
                           'end_station_id',\
                           'bike_id'],\
                   axis=1)

In [15]:
weather['precipitation_inches'] = weather['precipitation_inches'].apply(lambda x : 0.005 if (x == 'T') else float(x))

In [16]:

weather = weather[weather.precipitation_inches.isnull() == False]

weather = weather.fillna(0)

weather['events'] = weather['events'].apply(lambda x : 'Normal' if x == 0 else x)
weather['events'] = weather['events'].apply(lambda x : 'Rain' if x == 'rain' else x)
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [18]:
weather.dtypes

date                               object
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    float64
min_sea_level_pressure_inches     float64
max_visibility_miles              float64
mean_visibility_miles             float64
min_visibility_miles              float64
max_wind_Speed_mph                float64
mean_wind_speed_mph               float64
max_gust_speed_mph                float64
precipitation_inches              float64
cloud_cover                       float64
events                             object
wind_dir_degrees                  float64
zip_code                          

In [19]:
# PreProcessing trips

In [20]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


In [21]:
train['time'] = train['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )
test['time'] = test['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

In [22]:
train.loc[:,'start_date'] = train['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))
test.loc[:,'start_date'] = test['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))


In [23]:
train.loc[:,'day_of_week'] = train['start_date'].apply(lambda x : x.dayofweek )
test.loc[:,'day_of_week'] = test['start_date'].apply(lambda x : x.dayofweek )

In [24]:
train.loc[:,'month'] = train['start_date'].apply(lambda x : x.month )
test.loc[:,'month'] = test['start_date'].apply(lambda x : x.month )

In [25]:
train.loc[:,'year'] = train['start_date'].apply(lambda x : x.year )
test.loc[:,'year'] = test['start_date'].apply(lambda x : x.year )

In [26]:
train.loc[:,'dayofyear'] = train['start_date'].apply(lambda x : x.dayofyear )
test.loc[:,'dayofyear'] = test['start_date'].apply(lambda x : x.dayofyear )

In [27]:
train.loc[:,'start_date'] = train['start_date'].apply(lambda x : pd.to_datetime(x).date())
test.loc[:,'start_date'] = test['start_date'].apply(lambda x : pd.to_datetime(x).date())


In [28]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)

In [29]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [30]:
train = pd.merge(holiday_festive_day,train,on=['start_date'],how='right')
test = pd.merge(holiday_festive_day,test,on=['start_date'],how='right')
test.shape

(119998, 11)

In [32]:
train.loc[:,'is_holiday'] = train.holiday.isnull()
test.loc[:,'is_holiday'] = test.holiday.isnull()

In [33]:
train.loc[:,'is_holiday'] = train['is_holiday'].apply(lambda x : not x )
test.loc[:,'is_holiday'] = test['is_holiday'].apply(lambda x : not x )
test.shape

(119998, 12)

In [34]:
train.loc[:,'subscription_type'] = train['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
test.loc[:,'subscription_type'] = test['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
test.shape

(119998, 12)

In [35]:
#Merging dataframes

In [40]:
stationProcesed.rename(columns={'id':'start_station_id'},inplace=True)


In [41]:
train2 = pd.merge(train,\
                  stationProcesed,\
                  on= ['start_station_id'],\
                  how='inner')

test2 = pd.merge(test,\
                  stationProcesed,\
                  on= ['start_station_id'],\
                  how='inner')


In [45]:
# Preprocessing trips

In [54]:
train2 = train2.drop(labels=['holiday'],axis=1).dropna()

test2 = test2.drop(labels=['holiday'],axis=1).dropna()

In [55]:
train2.rename(columns={'start_date':'date'},inplace=True)
test2.rename(columns={'start_date':'date'},inplace=True)

In [56]:
# merging with weather, trainN where N - 1 count merges

In [57]:
train3 = pd.merge(train2,weather,\
                  on=['date','zip_code'],\
                  how='left')

test3 = pd.merge(test2,weather,\
                 on=['date','zip_code'],\
                 how='left')

In [59]:
train3.rename(columns={'start_station_id':'station_id'},inplace=True)

test3.rename(columns={'start_station_id':'station_id'},inplace=True)

In [76]:
# register with nan values in weather

In [142]:
nan_rows_test = test3[test3.isnull().T.any().T]
nan_rows_test.shape# 3 registers


,date,id,start_station_name,station_id,subscription_type,time,day_of_week,month,year,dayofyear,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees


In [143]:
train3
nan_rows_train = train3[train3.isnull().T.any().T]
nan_rows_train#11 registers

,date,id,duration,start_station_name,station_id,subscription_type,time,day_of_week,month,year,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees


In [ ]:
# zeros in nan values'test3

In [ ]:
train3 = train3.dropna()
test3 = test3.fillna(0)

In [90]:
test3.shape

(119998, 34)

In [92]:
train3.shape

(549950, 35)

In [136]:
# ML robert


In [137]:
train3.dtypes

date                               object
id                                  int64
duration                            int64
start_station_name                 object
station_id                          int64
subscription_type                   int64
time                                int64
day_of_week                         int64
month                               int64
year                                int64
dayofyear                           int64
is_holiday                           bool
zip_code                            int64
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    

In [160]:
import sklearn.preprocessing
def numberEvents(df):
    df['events'] = df.events.apply(eventNumber)

def eventNumber(eventString):
    if eventString == 'Normal' or eventString == 0: return 0
    elif eventString == 'Rain': return 1
    elif eventString == 'Fog': return 2
    elif eventString == 'Fog-Rain': return 3
    else: return 4
    
def cleanEvents(df):
    df['events'] = df.events.apply(lambda x: "Normal" if pd.isnull(x) else x)
    df['events'] = df.events.apply(lambda x: "Rain" if x == "rain" else x)

def encode(df):
    encoder = sklearn.preprocessing.OneHotEncoder()
    return pd.DataFrame(encoder.fit_transform(df).toarray())

In [161]:
trainCategories = train3.loc[:,['station_id',\
                        'subscription_type',
                       'day_of_week',\
                       'year',\
                       'month',\
                       'is_holiday',
                        'events'\
                       ]]

In [162]:
numberEvents(trainCategories)

In [163]:
trainCategories['events'].unique()

array([0, 2, 1, 3, 4])

In [164]:
trainCategories['is_holiday'].unique()

array([True, False], dtype=object)

In [165]:
trainCategories = encode(trainCategories)

In [175]:
trainNoCategories = train3.iloc[:,13:]
del trainNoCategories['events']


In [176]:
trainCategories.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [93]:
# probando el ML de braian


In [116]:
trainBR = train3[train3['duration'] < 5000]
trainBR.shape

(536049, 35)

In [95]:
test_id = test3['id']

In [117]:
train_target = trainBR['duration']

In [99]:
trainBR.dtypes

date                               object
id                                  int64
duration                            int64
start_station_name                 object
station_id                          int64
subscription_type                   int64
time                                int64
day_of_week                         int64
month                               int64
year                                int64
dayofyear                           int64
is_holiday                           bool
zip_code                            int64
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    

In [100]:
trainBR= trainBR.drop(labels=['duration','date','events','id'],axis=1)

In [101]:
testBR = test3.drop(labels=['date','events','id'],axis=1)

In [102]:
from sklearn.decomposition import IncrementalPCA

In [103]:
pca = IncrementalPCA(n_components=8)

In [106]:

del trainBR['start_station_name']
del testBR['start_station_name']

In [107]:
tripTransform = pca.fit_transform(trainBR)
tripTransform_test = pca.fit_transform(testBR)

In [108]:
tripTransform.shape

(536049, 8)

In [109]:
tripTransform_test.shape

(119998, 8)

In [110]:
tripTransformDF = pd.DataFrame(tripTransform)
tripTransformDF_test = pd.DataFrame(tripTransform_test)

In [121]:
trainBR = tripTransformDF
testBR = tripTransformDF_test

In [112]:
from sklearn import ensemble

In [113]:
rfr = ensemble.RandomForestRegressor(n_jobs=-1,n_estimators=50,criterion='mse',oob_score=True)

In [122]:
train_target.shape

(536049,)

In [123]:
trainBR.shape

(536049, 8)

In [124]:
rfr.fit(trainBR,train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [125]:
test_prediction = rfr.predict(testBR)

In [126]:
x = pd.read_csv('data/trip.csv',
                   usecols=['id','duration'])
target = pd.read_csv('data/trip_test.csv',
                   usecols=[ 'id'])
testTarget = pd.merge(x,target, how = "inner", on = "id")

In [127]:
test_prediction.shape

(119998,)

In [128]:
testTarget.shape

(119998, 2)

In [129]:
testTarget.dtypes

id          int64
duration    int64
dtype: object

In [130]:
from sklearn.metrics import mean_squared_error

In [132]:
mean_squared_error(testTarget['duration'], test_prediction )

42923231.095528767

In [133]:
testTarget['duration'] = test_prediction

In [135]:
testTarget.to_csv('data/prediction_1_kaggle.csv', index=False)